In [1]:
import pandas as pd

In [2]:
dataframe1 = pd.read_excel('Nginx - Airtel - Port Mapping - Port Forwarding.xlsx',sheet_name='Port-Mapping')

'''
for col in dataframe1.columns:
    print(col)
'''    

'\nfor col in dataframe1.columns:\n    print(col)\n'

In [3]:
# read each line, and generate nginx conf code
automatic_port = 50000
group_filter = 'ryzen9'
group_ip = '192.168.1.4'

upstream_text = ''
server_text = ''
nginx_conf_text = ''
for machine,ip,svc_name,i_port,p_port,group,is_forward,p_binding,fw_added in dataframe1.values:
    rule_needed = True
    public_port = 0
    
    # Filter for a specific Linux Host Nginx Conf
        # Generate entry only when not host machine
    if group == group_filter and machine != group:        
        
        # remove special characters
        if machine.find("-") > 0:
            machine = machine.replace('-','_')
        
        if p_port in ('Public-Full-Automatic','Local-Automatic'):
            automatic_port += 1
            public_port = automatic_port
            
        if p_port == 'Public-Semi-Automatic':
            group_ip_last_no = group_ip.split('.')[-1]
            public_port = f"{group_ip_last_no}{i_port}"
        
        rule_name = f"{machine}__{svc_name}__{public_port}"

        if p_port != 'X':
            upstream_str = f"""
        upstream {rule_name} {{
            server {ip}:{i_port} weight=1;
        }}"""

            server_str = f"""
        server {{
            listen {public_port};
            proxy_pass {rule_name};
        }}"""

            #print(upstream_str)
            upstream_text += upstream_str
            #print(server_str)
            server_text += server_str
            
            nginx_conf_text = upstream_str + server_str
            print(nginx_conf_text)
            
            #break;
    
#print(upstream_text)
#print(server_text)


        upstream sqlmonitor__rdp__50001 {
            server 192.168.100.10:3389 weight=1;
        }
        server {
            listen 50001;
            proxy_pass sqlmonitor__rdp__50001;
        }

        upstream sqlmonitor__grafana__43000 {
            server 192.168.100.10:3000 weight=1;
        }
        server {
            listen 43000;
            proxy_pass sqlmonitor__grafana__43000;
        }

        upstream sqlmonitor__mssql__50002 {
            server 192.168.100.10:1433 weight=1;
        }
        server {
            listen 50002;
            proxy_pass sqlmonitor__mssql__50002;
        }

        upstream ubuntu__ssh__50003 {
            server 192.168.100.22:22 weight=1;
        }
        server {
            listen 50003;
            proxy_pass ubuntu__ssh__50003;
        }

        upstream ubuntu__vnc__50004 {
            server 192.168.100.22:5800 weight=1;
        }
        server {
            listen 50004;
            proxy_pass ubuntu__vnc__50004;
      